## Using an automated approach to coregistration

In [100]:
from pathlib import Path
import os, glob 

# set the path

source_dir = os.path.join(Path(os.getcwd()).parent.parent.parent, 'nshor', 'Multiband_with_MEG')
# print(source_dir)
subjects_dir = os.path.join(Path(os.getcwd()).parent.parent, 'freesurfer', 'subjects')

# print(subjects_dir)
out_dir = os.path.join(Path(os.getcwd()).parent.parent, 'meg_coreg_raw')

# print(out_dir)

if not os.path.exists(out_dir):
    os.makedirs(out_dir)


## Load required packages

In [101]:
import numpy as np

import mne
from mne.coreg import Coregistration
from mne.io import read_info

## Setting fiducials 
It is to take the MEG sensor space (physical locations) to the  ``head`` coordinate frame (digitized locations))

For more information, see [here](https://mne.tools/1.1/auto_tutorials/forward/20_source_alignment.html)

In [102]:
sub01_fid = np.array([[-72.90, -2.57, -63.73],[-0.49, 87.38, -53.45],[75.54, -3.75, -63.74]]) #LPA, Nasion, RPA
sub02_fid = np.array([[-69.48, -2.53, -64.51],[0.49, 85.86, -51.75],[74.83, 3.05, -62.40]])
sub03_fid = np.array([[-74.59, -5.49, -64.53],[-1.54, 90.67, -53.76],[80.37, -4.51, -66.42]])
sub04_fid = np.array([[-76.96, 1.07, -71.03],[1.50, 88.75, -53.75],[81.86, -4.60, -68.93]])
sub05_fid = np.array([[-72.97, -1.70, -65.11],[1.63, 92.95, -51.98],[76.08, -4.46, -64.24]])
sub06_fid = np.array([[-76.17, -5.5, -60.15],[-0.56, 89.51, -41.6],[79.06, -8.84, -64.98]])
sub07_fid = np.array([[-73.79, -6.67, -63.82],[2.51, 87.14, -53.10],[78.87, -3.93, -57.46]])
sub08_fid = np.array([[-78.68, 5.50, -65.51],[3.49, 90.70, -53.68],[80.82, -0.45, -60.79]])
sub09_fid = np.array([[-71.48, 1.57, -71.50],[2.47, 88.66, -50.76],[73.32, 3.54, -71.47]])
sub10_fid = np.array([[-80.44, -8.52, -50.56],[-3.37, 89.98, -32.02],[84.89, 5.83, -44.76]])
sub11_fid = np.array([[-74.63, 5.59, -63.42],[3.12, 87.01, -47.96],[77.48, 0.51, -65.50]])
sub12_fid = np.array([[-77.75, -1.49, -70.57],[1.48, 92.32, -52.38],[84.13, 7.39, -62.08]])

In [103]:
# for subject in ['sub01','sub02','sub03','sub04','sub05','sub06',
#                 'sub07','sub08','sub09','sub10','sub11','sub12']:
#     for sess in range(60):

subject = 'sub-01'
sess = 1
sess = str(sess+1)
print(sess)

fname_raw = os.path.join(source_dir, subject, 'meg', f'{subject}_task-RDR_run-{sess}_meg.fif')
print(fname_raw)
info = read_info(fname_raw)
plot_kwargs = dict(subject=subject, subjects_dir=subjects_dir,
                    surfaces="head-dense", dig=True, eeg=[],
                    meg='sensors', show_axes=True,
                    coord_frame='meg')
view_kwargs = dict(azimuth=45, elevation=90, distance=0.6,
                    focalpoint=(0., 0., 0.))

2
/data/users2/nshor/Multiband_with_MEG/sub-01/meg/sub-01_task-RDR_run-2_meg.fif


## Set up the coregistration model

In [104]:
# FreeSurfer Talairach transformation file does not exist: 
fiducials = mne.coreg.get_mni_fiducials(subject, subjects_dir=subjects_dir)
# fiducials = 'estimated'
print(fiducials)
sub_fid = 'sub'+subject[3:]+'_fid'
new_id = sub_fid.replace('-', '')
print(new_id)
for i in range(3):
    fiducials[i]['r'] = locals()[new_id][i, :]*0.001
print(fiducials)

[<DigPoint |        LPA : (-79.5, -17.4, -4.6) mm   : MRI (surface RAS) frame>, <DigPoint |     Nasion : (1.2, 78.5, 31.5) mm      : MRI (surface RAS) frame>, <DigPoint |        RPA : (80.7, -9.8, -10.6) mm    : MRI (surface RAS) frame>]
sub01_fid
[<DigPoint |        LPA : (-72.9, -2.6, -63.7) mm   : MRI (surface RAS) frame>, <DigPoint |     Nasion : (-0.5, 87.4, -53.5) mm    : MRI (surface RAS) frame>, <DigPoint |        RPA : (75.5, -3.8, -63.7) mm    : MRI (surface RAS) frame>]


### Create head model

In [88]:
# This function will only work if freesurfer environment is set up and linked to this script
# but it is still unknown, so used the command line tool for BEM

# mne.bem.make_watershed_bem(  # for T1; for FLASH, use make_flash_bem instead
#     subject=subject,
#     subjects_dir=subjects_dir,
#     copy=True,
#     overwrite=True,
#     show=True,
# )

In [89]:
# Now create the scalp surfaces
# mne.bem.make_scalp_surfaces(
#     subject=subject,
#     subjects_dir=subjects_dir,
#     no_decimate=True,
#     force=True,
#     overwrite=True,
# )



In [105]:
#!pip install pyvistaqt

# fiducials = "estimated"  # get fiducials from fsaverage
coreg = Coregistration(info, subject, subjects_dir, fiducials=fiducials)
# fig = mne.viz.plot_alignment(info, trans=coreg.trans, **plot_kwargs)

    Triangle neighbors and vertex normals...
Using low resolution head model in /data/users2/mrahman21/freesurfer/subjects/sub-01/bem/outer_skin.surf
    Triangle neighbors and vertex normals...


## Initial fit with Fiducials 

Do first a coregistration fit using only 3 fiducial points. This allows to find a good initial solution before further optimization using head shape points. 

In [106]:
coreg.fit_fiducials(verbose=True)
# fig = mne.viz.plot_alignment(info, trans=coreg.trans, **plot_kwargs)

Aligning using fiducials
Start median distance:  17.91 mm
End   median distance:  14.89 mm


## Refining with ICP

Do this refinement using Iterative Closest Point (ICP) algorithm if initial fiducials are obtained from fsaverage and not from precise manual picking in the GUI 

In [107]:
coreg.fit_icp(n_iterations=6, nasion_weight=2., verbose=True)
# fig = mne.viz.plot_alignment(info, trans=coreg.trans, **plot_kwargs)

Aligning using ICP
Start     median distance:  14.89 mm
  ICP  1  median distance:  14.57 mm
  ICP  2  median distance:  13.02 mm
  ICP  3  median distance:  12.04 mm
  ICP  4  median distance:  10.72 mm
  ICP  5  median distance:   8.88 mm
  ICP  6  median distance:   7.58 mm
End       median distance:   7.58 mm


## Omit Bad Points

Set a threshold (distance) above which all points are considered outliers 

In [108]:
coreg.omit_head_shape_points(distance=5. / 1000)  # distance is in meters

Coregistration: Excluding 92 head shape points with distance >= 0.005 m.


# Final fit

In [109]:
coreg.fit_icp(n_iterations=20, nasion_weight=10., verbose=True)
# fig = mne.viz.plot_alignment(info, trans=coreg.trans, **plot_kwargs)
# mne.viz.set_3d_view(fig, **view_kwargs)

dists = coreg.compute_dig_mri_distances() * 1e3  # in mm
print(
    f"Distance between HSP and MRI (mean/min/max):\n{np.mean(dists):.2f} mm "
    f"/ {np.min(dists):.2f} mm / {np.max(dists):.2f} mm"
)

Aligning using ICP
Start     median distance:   3.20 mm
  ICP  1  median distance:   3.10 mm
  ICP  2  median distance:   2.58 mm
  ICP  3  median distance:   2.66 mm
  ICP  4  median distance:   2.19 mm
  ICP  5  median distance:   2.14 mm
  ICP  6  median distance:   2.09 mm
  ICP  7  median distance:   2.05 mm
End       median distance:   2.05 mm
Distance between HSP and MRI (mean/min/max):
3.53 mm / 0.62 mm / 14.83 mm


## Save the final ``trans`` file

In [110]:
out_path = os.path.join(out_dir, subject)
if not os.path.exists(out_path):
    os.makedirs(out_path)
mne.write_trans(out_path + f'/{subject}_task-RDR_run-{sess}_trans.fif', coreg.trans, overwrite=True)
coreg.trans

# mne.write_trans(ROOT + 'meg_pku_raw/'+subject+'/run'+sess+'_trans.fif', coreg.trans)

<Transform | head->MRI (surface RAS)>
[[ 0.99901167  0.02922153 -0.03349314  0.00164562]
 [-0.02729693  0.99802674  0.05654654 -0.03883669]
 [ 0.03507943 -0.0555764   0.99783801 -0.02529462]
 [ 0.          0.          0.          1.        ]]